<a href="https://colab.research.google.com/github/wingated/cs474_labs_f2019/blob/master/DL_Lab2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab 2: Intro to PyTorch

## Deliverable

For this lab, you will submit an ipython notebook via learningsuite.
This lab will be mostly boilerplate code, but you will be required to implement a few extras.

**NOTE: you almost certainly will not understand most of what's going on in this lab!
That's ok - the point is just to get you going with pytorch.
We'll be working on developing a deeper understanding of every part of this code
over the course of the next two weeks.**

A major goal of this lab is to help you become conversant in working through pytorch
tutorials and documentation.
So, you should feel free to google whatever you want and need!

This notebook will have three parts:

* Part 1: Your notebook should contain the boilerplate code. See below.

* Part 2: Your notebook should extend the boilerplate code by adding a testing loop.

* Part 3: Your notebook should extend the boilerplate code by adding a visualization of test/training performance over time.

The resulting image could, for example, look like this:
![](http://liftothers.org/dokuwiki/lib/exe/fetch.php?cache=&w=900&h=608&tok=3092fe&media=cs501r_f2018:lab2.png)
See the assigned readings for pointers to documentation on pytorch.
___

### Grading standards:
Your notebook will be graded on the following:

* 50% Successfully followed lab video and typed in code
* 20% Modified code to include a test/train split
* 20% Modified code to include a visualization of train/test losses
* 10% Tidy and legible figures, including labeled axes where appropriate
___

### Description
Throughout this class, we will be using pytorch to implement our deep neural networks. 
Pytorch is a deep learning framework that handles the low-level details of 
GPU integration and automatic differentiation.

The goal of this lab is to help you become familiar with pytorch. 
The three parts of the lab are outlined above.

For part 1, you should watch the video below, and type in the code as it is explained to you.

A more detailed outline of Part 1 is below.

For part 2, you must add a validation (or testing) loop using the 
FashionMNIST dataset with train=False

For part 3, you must plot the loss values.

Optional: Demonstrate overfitting on the training data.

The easiest way to do this is to limit the size of your training dataset 
so that it only returns a single batch (ie len(dataloader) == batch_size, 
and train for multiple epochs. For example,
I set my batch size to 42, and augmented my dataloader to produce only 42 
unique items by overwriting the len function to return 42. 
In my training loop, I performed a validation every epoch which basically corresponded 
to a validation every step.

In practice, you will normally compute your validation loss every n steps, 
rather than at the end of every epoch. This is because some epochs can take hours, 
or even days and you don’t often want to wait that long to see your results.

Testing your algorithm by using a single batch and training until overfitting 
is a great way of making sure that your model and optimizer are working the way they should!

___

### Part 0
Watch Tutorial Video

[https://youtu.be/0P-YctShbwc](https://youtu.be/0P-YctShbwc)

**TODO:**

**DONE:**
* Watch video
___

### Part 1
Your notebook should contain the boilerplate code. See below.

**TODO:**

**DONE:**
* Replicate boilerplate from the video
___

### Part 2
Your notebook should extend the boilerplate code by adding a testing loop.

**TODO:**

**DONE:**
* Add a testing (validation) loop

In [ ]:
!pip3 install torch 
!pip3 install torchvision
!pip3 install tqdm

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
import matplotlib.pyplot as plt
from torchvision import transforms, utils, datasets
from tqdm import tqdm
 
assert torch.cuda.is_available() # You need to request a GPU from Runtime > Change Runtime Type

# Write the boilerplate code from the video here
class LinearNetwork(nn.Module):
  def __init__(self, dataset, in_dim=784, out_dim=10):
    super(LinearNetwork, self).__init__()
    self.net = nn.Sequential(nn.Linear(in_dim, out_dim))
    
  def forward(self, x):
    n,c,h,w = x.size()
    flattened = x.view(n, c*h*w)
    return self.net(flattened)
    
# Create a dataset class that extends the torch.utils.data Dataset class here
class FashionMNISTProcessedDataset(Dataset):
  def __init__(self, root, train=True):
    self.data = datasets.FashionMNIST(root,
                                     train=train,
                                     transform=transforms.ToTensor(),
                                     download=True)
    
  def __getitem__(self, i):
    x, y = self.data[i]
    return x, y
      
  def __len__(self):
    return len(self.data)

# Extend the torch.Module class to create your own neural network

# Instantiate the train and validation sets
train_dataset = FashionMNISTProcessedDataset('/tmp/fashionmnist', train=True)
val_dataset = FashionMNISTProcessedDataset('/tmp/fashionmnist', train=False)

# Instantiate your data loaders
train_loader = DataLoader(train_dataset, batch_size=42, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=42, pin_memory=True)

# Instantiate your model and loss and optimizer functions
x,y = train_dataset.data[0]
c,h,w = x.size()
model = LinearNetwork(train_dataset, in_dim=c*h*w)
model = model.cuda()
objective = torch.nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=1e-4)

# Run your training / validation loops
train_losses = []
val_losses = []

#print('length of training: ',len(train_loader))
#print('length of validation: ',len(val_loader))

num_epochs = 500
for epoch in range(num_epochs):
  train_loop = tqdm(total=len(train_loader), position=0)
  batch_losses = []
  for x, y_truth in train_loader:
    x, y_truth = x.cuda(async=True), y_truth.cuda(async=True)

#    print('x size: ',x.size())
#    print('y size: ',y_truth.size())
    
    optimizer.zero_grad()
    y_hat = model(x)
    loss = objective(y_hat, y_truth)
    batch_losses.append(loss)

    train_loop.set_description('epoch: {}, train loss: {:.4f}'.format(epoch,loss.item()))
    train_loop.update(1)
    
    loss.backward()
    optimizer.step()
  
  avg_batch_loss = sum(batch_losses) / len(batch_losses)
  train_losses.append(avg_batch_loss)
  
  train_loop.close()
  
  val_loop = tqdm(total=len(val_loader), position=0)
  batch_losses = []
  with torch.no_grad():
    for val_x, val_y_truth in val_loader:
      val_x, val_y_truth = val_x.cuda(async=True), val_y_truth.cuda(async=True)
#      print('x_val size: ', val_x.size())
#      print('y_val size: ', val_y_truth.size())
    
      val_y_hat = model(val_x)
      loss = objective(val_y_hat, val_y_truth)
      batch_losses.append(loss)
    
      val_loop.set_description('epoch: {}, val loss: {:.4f}'.format(epoch,loss.item()))
      val_loop.update(1)
    
  avg_batch_loss = sum(batch_losses) / len(batch_losses)
  val_losses.append(avg_batch_loss)
  val_loop.close()



___

### Part 3
Your notebook should extend the boilerplate code by adding a visualization of test/training
performance over time. Use matplotlib.pyplot

**TODO:**

**DONE:**
* Add a visualization of test/train performance (i.e. loss) over time.

In [ ]:
# Write your code to create a plot of your loss over time
plt.figure(1)
plt.plot(train_losses, 'b', label='training')
plt.plot(val_losses, 'r', label='validation')
plt.legend()
plt.xlabel('Epoch')
plt.ylabel('Cross Entropy Loss')
plt.title('Lab 2 Training & Test Performance')
plt.show()